In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
from collections import Counter
import re
import plotly.express as px
import datetime

In [2]:
arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_84522/777748067.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")


In [3]:
def clean_themes(themes):
    if type(themes) == str:
        themes = themes.split(',')
        themes = [theme.strip() for theme in themes]
        themes = [theme.lower() for theme in themes]
        return themes
    return []

arcom_df['clean_theme'] = arcom_df['Thème'].apply(clean_themes)
arcom_df['clean_date'] = [datetime.datetime.strptime(x, '%d/%m/%Y').date() for x in arcom_df['Date']]

In [27]:
drinks_df = arcom_df[arcom_df['Secteur'] == 'BOISSONS']
drinks_df = drinks_df.reset_index()

# diff to diff

In [28]:
drinks_df['Variété'].value_counts()

BOISSONS AUX FRUITS                184
BOISSONS AU THE                    121
EAUX GAZEUSES                       93
EAUX MINERALES PLATES               91
BOISSONS COLA                       79
BOISSONS ENERGISANTES               76
JUS DE FRUITS AMBIANTS              64
BOISSONS COLA ALLEGES               54
SODAS                               49
EAUX DE SOURCE PLATES               45
BIERES SANS ALCOOL                  40
SOFT DRINKS GAMME                   40
EAUX PLATES AROMATISEES             39
TONIQUES                            38
EAUX GAZEUSES AROMATISEES           31
JUS DE FRUITS FRAIS                 30
INTERNET SOFT DRINKS                20
PRESTIGE SOFT DRINK                 20
CONCENTRES SUCRES & SIROP           17
CLEAR-LIMES                         13
MOUSSEUX PETILLANTS SANS ALCOOL     11
EAUX GAMME                           9
BOISSONS AU CAFE                     6
CONCENTRES SANS SUCRE                5
APERITIFS SANS ALCOOL                5
BOISSONS DIVERSES        

In [29]:
def clean_themes(themes):
    if type(themes) == str:
        themes = themes.split(',')
        themes = [theme.strip() for theme in themes]
        themes = [theme.lower() for theme in themes]
        return themes
    return []

drinks_df['clean_theme'] = drinks_df['Thème'].apply(clean_themes)
drinks_df['clean_date'] = [datetime.datetime.strptime(x, '%d/%m/%Y').date() for x in drinks_df['Date']]


In [30]:
most_frequent_themes = Counter([word for sublist in drinks_df['clean_theme'] for word in sublist])
most_frequent_themes.most_common()

[('gout', 355),
 ('typographie', 298),
 ('effet visuel', 247),
 ('effet', 241),
 ('bonne humeur', 224),
 ('environnement autre', 224),
 ('humour autre', 222),
 ('developpement durable', 204),
 ('musique celebre', 203),
 ('naturel', 193),
 ('comportement autre', 189),
 ('fraicheur', 180),
 ('joie', 179),
 ('anglais', 178),
 ('nouveaute autre', 173),
 ('interpellation', 171),
 ('nature', 164),
 ('musique autre', 160),
 ('composant special', 149),
 ('convivialite', 147),
 ('choix', 140),
 ('signature etrangere', 135),
 ('festif', 133),
 ('conditionnement', 131),
 ('animal autre', 129),
 ('etonnement', 128),
 ('sans parole', 126),
 ('amitie', 125),
 ('bruitage autre', 124),
 ('jeune', 124),
 ('tag', 122),
 ('interrogation', 119),
 ('bien-etre', 111),
 ('changement autre', 109),
 ('enfant', 107),
 ('univers urbain', 105),
 ('recyclage', 103),
 ('pensee positive', 102),
 ('changement de signature', 102),
 ('objet personnifie', 99),
 ('saisonnalite', 99),
 ('ludique', 91),
 ('tutoiement', 91)

In [31]:
words_to_plot = ['gout', 'bonne humeur', 'environnement autre', 'nutri-score']


def plot_theme_prevalence(theme=''):
    sub_df = drinks_df[[theme in x for x in drinks_df['clean_theme']]]
    fig = px.histogram(sub_df,
                x='clean_date',
                color='Secteur', title=f'Ads with {theme} in themes')
    fig.show()

for word in words_to_plot: plot_theme_prevalence(word)


In [32]:
token_pattern = r'\b(?:\w+-\w+|\w+|nutri-score)\b'
vectorizer = CountVectorizer(tokenizer =lambda t: re.findall(token_pattern, t.lower()))

In [33]:
water_variety_names = ['EAUX GAZEUSES', 'EAUX MINERALES PLATES']

drinks_df['group'] = ['group' if x in water_variety_names else 'treatment' for x in drinks_df['Variété']]

In [34]:
token_pattern = r'\b(?:\w+-\w+|\w+|nutri-score)\b'
vectorizer = CountVectorizer(token_pattern=token_pattern)


X = vectorizer.fit_transform(drinks_df['Thème'])
word_counts = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
word_counts

,a,absurde,accent,action,actualite,aeronautique,affichage,africain,agressivite,agricole,...,visuelle,vitalite,voeux,voix,volley-ball,vtt,vue,yoga,you,youtubeur
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1190,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1192,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:

#nutriscore_date='2017-06-01'

def estimate_DiD(target_word, date = datetime.date(2017, 6, 1)):
    data = pd.concat([drinks_df['clean_theme'], drinks_df['group'], word_counts], axis=1)
    data['outcome'] = data[target_word]
    data['prior'] = np.where(drinks_df['clean_date']<date, 1, 0)
    data['post'] = np.where(drinks_df['clean_date']>=date, 1, 0)
    data['treated'] = np.where(data['group'] == 'treatment', 1, 0)
    data['post_treated'] = data['post'] * data['treated']
    model = smf.ols('outcome ~ post + treated + post_treated', data=data).fit()
    print(model.summary())
    did_estimate = model.params['post_treated']
    print('\r-------------------------\r')
    print(f'DiD Estimate for the word {target_word}: {did_estimate}')
    print('\r-------------------------\r')

In [36]:
estimate_DiD('gout')

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.050
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     20.67
Date:                Tue, 16 Jul 2024   Prob (F-statistic):           4.68e-13
Time:                        12:49:00   Log-Likelihood:                -729.11
No. Observations:                1194   AIC:                             1466.
Df Residuals:                    1190   BIC:                             1487.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0133      0.052      0.259   